In [2]:
import pandas as pd
import matplotlib.pyplot as plt 


In [9]:
def restructure_df(df_date):
    
    df_rotate = df_date.T
    df_rotate.columns = df_rotate.iloc[0]
    df_rotate = df_rotate.iloc[1:]
    df_rotate.rename(columns={'Breakfast': 'date_day'}, inplace=True)
    
    df_rotate[['day', 'data']] = df_rotate['date_day'].str.split('-', expand=True)
    condensed_df = df_rotate.drop(columns=['date_day'])

    pattern = r'(\w+) (\d+)'
    condensed_df[['Month', 'Day']] = condensed_df['data'].str.extract(pattern)

    month_mapping = {
        'January': '01',
        'February': '02',
        'March': '03',
        'April': '04',
        'May': '05',
        'June': '06',
        'July': '07',
        'August': '08',
        'September': '09',
        'October': '10',
        'November': '11',
        'December': '12'
    }

    condensed_df['Month'] = condensed_df['Month'].map(month_mapping)
    condensed_df['Year'] = '2023'
    condensed_df['Date'] = condensed_df['Year'] + '-' + condensed_df['Month'] + '-' + condensed_df['Day']
    condensed_df['Date'] = pd.to_datetime(condensed_df['Date'], format='%Y-%m-%d') 
    condensed_df.drop(columns=['Year', 'Month', 'Lunch', 'Dinner'], inplace=True)
    condensed_df.index = [x for x in range(len(condensed_df))]

    last_column = condensed_df.pop('Date')
    new_df = pd.concat([last_column, condensed_df], axis=1)
    
    return new_df

In [11]:
import pandas as pd

# Define a list of file names
import pandas as pd

# Assuming you have defined the 'restructure_df' function

file_names = [
    '24th July  Menu.xlsx',
    'Aug 7 Menu with calories.xlsx',
    'Copy of Plaksha menu from 28 August.xlsx',
    'Menu from 26th June.xlsx',
    'Menu July 3 (with calories).xlsx',  
    'Menu July 9.xlsx',
    'menu week 19june to 25june 2023 (2).xlsx',
    'Plaksha menu August14 with calorie-count, 2023..xlsx',
    'Plaksha menu from 4 Sept with calorie count.xlsx',
    'Plaksha MEnu from 8th May.xlsx',
    'Plaksha menu from 15May.xlsx', 
    'Plaksha menu from 17 july 2023 (1).xlsx', 
    'Plaksha Menu from May 1.xlsx', 
    'Plaksha menu starting from Sept 11, 2023..xlsx',
    'Plaksha menu starting from Sept 18, 2023..xlsx'
]

menu_dataframes = {}
for file_name in file_names:
    menu_df = pd.read_excel(file_name)
    restructured_df = restructure_df(menu_df)  
    menu_dataframes[file_name] = restructured_df

# Save the dataframes as CSV files
for file_name, dataframe in menu_dataframes.items():
    csv_file_name = file_name.replace('.xlsx', '.csv')  # Change file extension to CSV
    dataframe.to_csv(csv_file_name, index=False)


'''for file_name, dataframe in menu_dataframes.items():
    print(f"Dataframe from file: {file_name}")
    print(dataframe)

for key, df in menu_dataframes.items():
    if 'Date' in df.columns:
        df.set_index('Date', inplace=True)
    else:
        continue   
combined_menu_df = pd.concat(menu_dataframes.values(), ignore_index=True)'''




'for file_name, dataframe in menu_dataframes.items():\n    print(f"Dataframe from file: {file_name}")\n    print(dataframe)\n\nfor key, df in menu_dataframes.items():\n    if \'Date\' in df.columns:\n        df.set_index(\'Date\', inplace=True)\n    else:\n        continue   \ncombined_menu_df = pd.concat(menu_dataframes.values(), ignore_index=True)'

In [30]:
import pandas as pd
from dateutil import parser

file_names = [
    '24th July  Menu.csv',
    'Aug 7 Menu with calories.csv',
    'Copy of Plaksha menu from 28 August.csv',
    'Menu from 26th June.csv',
    'Menu July 3 (with calories).csv',  
    'Menu July 9.csv',
    'menu week 19june to 25june 2023 (2).csv',
    'Plaksha menu August14 with calorie-count, 2023..csv',
    'Plaksha menu from 4 Sept with calorie count.csv',
    'Plaksha MEnu from 8th May.csv',
    'Plaksha menu from 15May.csv', 
    'Plaksha menu from 17 july 2023 (1).csv', 
    'Plaksha Menu from May 1.csv', 
    'Plaksha menu starting from Sept 11, 2023..csv',
    'Plaksha menu starting from Sept 18, 2023..csv'
]

merged_data = pd.DataFrame()
for file_name in file_names:
    df = pd.read_csv(file_name)
    df['Date'] = df['Date'].apply(lambda x: parser.parse(x))
    merged_data = pd.concat([merged_data,df], ignore_index=True)

In [ ]:

merged_data.index = [x for x in range(len(merged_data))]
print(merged_data)
merged_data.to_csv('menu_final.csv')

In [42]:
new_dataset = pd.read_csv('merged_file.csv')
new_dataset['Date'] = pd.to_datetime(new_dataset['Date'], format='%d-%m-%Y %H:%M', errors='coerce')
dataset = new_dataset.drop(index=range(92, 99))
concat_dataset = pd.merge(new_dataset, merged_data, on='Date')

concat_dataset.to_csv('horrid.csv')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load your dataset
dataset = pd.read_csv('horrid.csv')
summary_stats = dataset.describe()

# Data Visualization
plt.figure(figsize=(12, 6))

# Histograms for each meal
meal_columns = ['B.FAST', 'LUNCH','Snacks']
for meal in meal_columns:
    plt.subplot(2, 2, meal_columns.index(meal) + 1)
    plt.hist(dataset[meal], bins=20, edgecolor='k')
    plt.title(f'{meal} Footfall Distribution')
    plt.xlabel('Footfall')
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Correlation Analysis
correlation_matrix = dataset[meal_columns].corr()

# Box Plots for each meal
dataset[meal_columns].boxplot()
plt.title('Box Plots for Meal Footfall')
plt.ylabel('Footfall')
plt.show()

# Outlier Detection (assuming 'Breakfast' as an example)
q1 = dataset['LUNCH'].quantile(0.25)
q3 = dataset['LUNCH'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr
outliers = dataset[(dataset['LUNCH'] < lower_bound) | (dataset['LUNCH'] > upper_bound)]




In [53]:
from sklearn.preprocessing import LabelEncoder
for column in dataset.columns:
    dataset[column] = label_encoder.fit_transform(dataset[column])
breakfast_correlations = dataset.corrwith(dataset['B.FAST'])

In [ ]:
import seaborn as sns 
correlation_matrix = dataset.corr()

# Create a heatmap
plt.figure(figsize=(30, 20))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title("Correlation Matrix")
plt.show()